## Setup

In [82]:
# Setup
import geopandas as gpd
import flopy
import numpy as np
import os
import pandas as pd

In [83]:
cwd = '\\'.join(os.getcwd().split('\\')[:-2])
cwd

'c:\\repos\\hm\\MAURICE'

## Tests

### Test: Load the model

In [3]:
# bug detected:
# 1. if the script is not in the same folder as the .nam file,
#    an error arises saying that the ._kx file is not found
# 2. if the .sfr file is in a SFR2 format (e.g. NSRTM = 2),
#    the file doesn't get read

m = flopy.mfusg.MfUsg.load(os.path.join(os.getcwd(), 't_m11_C1.nam'))

c:\Users\user\anaconda3\envs\maurice\Lib\site-packages\flopy\mbase.py:97: UserWarning: The program mfusg does not exist or is not executable.
  warn(


In [5]:
m.sfr.reach_data

rec.array([(4552, 0, 0, 0, 1,  1, 106., 0., 0., 0., 0., 0., 0., 0., 0.,   1,   2),
           (4553, 0, 0, 0, 1,  2,   8., 0., 0., 0., 0., 0., 0., 0., 0.,   2,   3),
           (4676, 0, 0, 0, 1,  3,  99., 0., 0., 0., 0., 0., 0., 0., 0.,   3,   4),
           (4677, 0, 0, 0, 1,  4,  63., 0., 0., 0., 0., 0., 0., 0., 0.,   4,   5),
           (4678, 0, 0, 0, 2,  1,  99., 0., 0., 0., 0., 0., 0., 0., 0.,   5,   6),
           (4679, 0, 0, 0, 2,  2,  97., 0., 0., 0., 0., 0., 0., 0., 0.,   6,   7),
           (4802, 0, 0, 0, 2,  3,   4., 0., 0., 0., 0., 0., 0., 0., 0.,   7,   8),
           (4803, 0, 0, 0, 2,  4, 100., 0., 0., 0., 0., 0., 0., 0., 0.,   8,   9),
           (4804, 0, 0, 0, 2,  5, 106., 0., 0., 0., 0., 0., 0., 0., 0.,   9,  10),
           (4805, 0, 0, 0, 2,  6,  88., 0., 0., 0., 0., 0., 0., 0., 0.,  10,  11),
           (4928, 0, 0, 0, 2,  7,  32., 0., 0., 0., 0., 0., 0., 0., 0.,  11,  12),
           (4929, 0, 0, 0, 2,  8, 122., 0., 0., 0., 0., 0., 0., 0., 0.,  12,  13),
    

In [18]:
# m.sfr.reach_data

In [33]:
# m.sfr.segment_data

In [ ]:
sp = 1 #stress period number
m.sfr.segment_data[sp-1]

In [12]:
m.sfr.nstrm

np.int64(174)

In [14]:
m.sfr.isfropt

0

In [ ]:
m.sfr.isfropt = 2
#setting it from here doesn't change the output format

#### Test: load sfr condition from shapefile

In [109]:
sfr_shp = gpd.read_file(os.path.join(cwd, 'test_models', 'SFR_ICALC2.shp'))

In [ ]:
sfr_shp

,row,column,layer,node,Reach,Stage,Bottom,Conductanc,Conc,Flow,...,width73,head74,flow74,width74,head75,flow75,width75,Flux_gw,Flux_S,geometry
0,38,2,1,4553,1,178.725368,178.594126,0.003604,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,"POLYGON ((490818.273 5043139.096, 490909.627 5..."
1,38,3,1,4554,2,178.723901,178.592659,0.000272,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,"POLYGON ((490909.627 5043179.77, 491000.982 50..."
2,39,3,1,4677,3,178.713966,178.582724,0.003366,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,"POLYGON ((490950.301 5043088.415, 491041.656 5..."
3,39,4,1,4678,4,178.699990,178.568748,0.002142,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,"POLYGON ((491041.656 5043129.089, 491133.01 50..."
4,39,5,1,4679,1,178.684601,178.553359,0.003366,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,"POLYGON ((491133.01 5043169.763, 491224.365 50..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,57,105,1,6993,5,171.908132,171.776890,0.003468,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,"POLYGON ((501000.591 5045592.747, 501091.945 5..."
170,57,106,1,6994,6,171.838517,171.707274,0.003400,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,"POLYGON ((501091.945 5045633.421, 501183.3 504..."
171,57,107,1,6995,7,171.769374,171.638131,0.003400,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,"POLYGON ((501183.3 5045674.095, 501274.654 504..."
172,57,108,1,6996,8,171.689851,171.558609,0.003434,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,"POLYGON ((501274.654 5045714.768, 501366.009 5..."


In [8]:
sfr_shp.Slope

0      0.000019
1      0.000019
2      0.000019
3      0.000019
4      0.000019
         ...   
169    0.000019
170    0.000019
171    0.000019
172    0.000019
173    0.000019
Name: Slope, Length: 174, dtype: float64

### Test: Modify the segment data parameters

In [3]:
# Load the data to be set up into the different stress periods
q = pd.read_csv(os.path.join(cwd, 'test_models', 'portate_segmenti.csv'))
q[q.isna()] = 0

In [ ]:
for sp in range(1, 91+1):
    tool = q.loc[q.SP == sp, ['Qin', 'Sg_2', 'Sg_3', 'Sg_4', 'Sg_5', 'Sg_6', 'Sg_7']].values[0]
    tool = np.around(tool, 5)
    tool = [tool[0]] + [-x if x != 0 else 0 for x in tool[1:]]
    m.sfr.segment_data[sp-1].flow = tool

### Test: write the new .sfr file

In [16]:
m.model_ws = os.path.join(cwd, 'test_models', 'tool_m11_c1', 'test_datasets_m11_c1')

In [19]:
m.write_input()

## Test: load reach and segment data from Excel file

In [185]:
# Load a tool model (with SFR package in a format readable by flopy)
m = flopy.mfusg.MfUsg.load(os.path.join(os.getcwd(), 't_m11_C1.nam'))

c:\Users\user\anaconda3\envs\maurice\Lib\site-packages\flopy\mbase.py:97: UserWarning: The program mfusg does not exist or is not executable.
  warn(


In [186]:
m.sfr


Streamflow-Routing (SFR2) Package Class

Parameters
----------
model : model object
    The model object (of type :class:'flopy.modflow.mf.Modflow') to which
    this package will be added.
nstrm : integer
    An integer value that can be specified to be positive or negative. The
    absolute value of NSTRM is equal to the number of stream reaches
    (finite-difference cells) that are active during the simulation and
    the number of lines of data to be included in Item 2, described below.
    When NSTRM is specified to be a negative integer, it is also used as a
    flag for changing the format of the data input, for simulating
    unsaturated flow beneath streams, and (or) for simulating transient
    streamflow routing (for MODFLOW-2005 simulations only), depending
    on the values specified for variables ISFROPT and IRTFLG, as described
    below. When NSTRM is negative, NSFRPAR must be set to zero, which means
    that parameters cannot be specified. By default, nstrm is set t

https://flopy.readthedocs.io/en/stable/Notebooks/sfrpackage_example.html

In [187]:
m.sfr.isfropt = 1
m.sfr.nrstrm = -174

In [188]:
m.sfr.write_file('test_v1.sfr')

In [162]:
# Load SFR characteristics from Excel files and shapefile
sfr_data = os.path.join(cwd, 'test_models', 'm11_C1_sfr_data.xlsx')
sfr_shp = gpd.read_file(os.path.join(cwd, 'test_models', 'SFR_ICALC2.shp'))

# Load general parameters (item 1)
it1 = pd.read_excel(sfr_data, sheet_name = 'ITEM1')

# Load reach data (item 2)
reach_data = pd.read_excel(sfr_data, sheet_name = 'ITEM2b')
reach_data = reach_data.merge(sfr_shp.loc[:, ['node', 'row', 'column', 'layer']], left_on = 'NRCH', right_on = 'node')
reach_data = reach_data.drop(columns = 'NRCH')
reach_data = reach_data.apply(pd.to_numeric)
reach_data.columns = ['iseg', 'ireach', 'rchlen', 'strtop', 'slope',  'strthick',  'strhc1', 'node', 'k', 'i', 'j']
reach_data = reach_data.loc[:,:].to_records(index = False)
# flopy adds 1 to node???
# reach_data.node = reach_data.node - 1

# Load item 5
it5 = pd.read_excel(sfr_data, sheet_name = 'ITEM5')

# Load segment data (item 6a)
segment_data = pd.read_excel(sfr_data, sheet_name = 'ITEM6abc')
segment_data.columns = [x.lower() for x in segment_data.columns]
segment_data = segment_data.loc[:,:].to_records(index = False)
segment_data = {0: segment_data}

# Set the unit number
unit_number = 27

# Load the data to be set up into the different stress periods
q = pd.read_csv(os.path.join(cwd, 'test_models', 'portate_segmenti.csv'))
q[q.isna()] = 0

In [177]:
m = flopy.modflow.Modflow(stress_periods = 91)
sfr = flopy.modflow.ModflowSfr2(
    m,
    nstrm = it1.NSTRM.values[0],              # number of reaches
    nss = it1.NSS.values[0],                  # number of segments
    const = it1.CONST.values[0],              # constant for manning's equation: 1 for m/s
    dleak = it1.DLEAK.values[0],              # closure tolerance for stream stage computation
    ipakcb = it1.ISTCB1.values[0],            # flag for writing SFR output to cell-by-cell budget (on unit 50)
    istcb2 = it1.ISTCB2.values[0],            # flag for writing SFR output to text file
    dataset_5 = {0: it5.values[0].tolist()},
    unit_number = unit_number,
    isfropt = it1.ISFROPT.values[0],          # ISFROPT parameter, defines input file format writing
    reach_data = reach_data
)

c:\Users\user\anaconda3\envs\maurice\Lib\site-packages\flopy\mbase.py:442: UserWarning: unhandled keywords: {'stress_periods': 91}
  warn(


In [179]:
# Insert the necessary flow values for each stress period
for sp in range(1, 91+1):
    tool = q.loc[q.SP == sp, ['Qin', 'Sg_2', 'Sg_3', 'Sg_4', 'Sg_5', 'Sg_6', 'Sg_7']].values[0]
    tool = np.around(tool, 5)
    tool = [tool[0]] + [-x if x != 0 else 0 for x in tool[1:]]
    segment_data[0].flow = tool
    sfr.segment_data[sp-1] = segment_data[0].copy()
    # sfr.stress_period_data[sp-1] = segment_data[0].copy()

In [180]:
# non stampa gli stress periods
sfr.write_file('test.sfr')

In [ ]:
sfr.stress_period_data

In [190]:
sfr.get_variable_by_stress_period('flow')

rec.array([(38, 2, 1, 1, 1, 1.18571)],
          dtype=[('k', '<i8'), ('i', '<i8'), ('j', '<i8'), ('iseg', '<i8'), ('ireach', '<i8'), ('flow0', '<f8')])